#### Including all the packages i need

In [1]:
import numpy as np # math package to work with array

import pandas as pd # data analysis package
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim # address to latitude and longitude

import requests
from pandas.io.json import json_normalize # json -> pandas

#Plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# package for clustering
from sklearn.cluster import KMeans

import folium # map rendering library
from termcolor import colored


print('All Packages are imported.')

All Packages are imported.


#### Read the data from the data source and get relevant data

In [2]:
nyc_data = open("dataset_TSMC2014_NYC.txt")

#i use dicts to get a relation between the store id´s and category
# and the user id & store id to count distinct visitors of the shops
places = dict()
categories = {}
coordinates = []
i=0
for v in nyc_data.readlines():
    venue = v.split("\t") #first tried to split with \n -> you should split with \t
    coordinates.append([venue[4],venue[5]]) #get the coordinates
    
    if venue[1] not in categories: #store id -> store category
        categories[venue[1]] = venue[3]
        
    if venue[1] not in places: # store id -> User ID, the length of the value corresponds to the nb of visitors
        places[venue[1]] = [venue[0]]
        
    if venue[0] not in places[venue[1]]:
        places[venue[1]].append(venue[0])
        

In [3]:
print("Venue with ID: 49bbd6c0f964a520f4531fe3 had ",len(places["49bbd6c0f964a520f4531fe3"]) ," visitors")
print("overall we have", len(places.keys()), "different venues")

Venue with ID: 49bbd6c0f964a520f4531fe3 had  7  visitors
overall we have 38333 different venues


#### Convert the stored data into pandas dataframe

In [6]:
cols = ["Venue ID", "Category", "Nb of Visitors", "Latidude" , "Longitude"]
df_venue = pd.DataFrame(columns=cols)

# i dont want to use all of the 38k stores, so i select the first 3000

for i in range(0,5000):
    Venue_ID = list(places.keys())[i] 
    coords = coordinates[i]
    Nb_Visitors = len(places[Venue_ID])
    category = categories[Venue_ID]
    df_venue = df_venue.append({"Venue ID":Venue_ID,
                                 "Category":category,
                                 "Nb of Visitors":Nb_Visitors,
                                 "Latidude":coords[0] ,
                                 "Longitude":coords[1]},ignore_index=True)

In [7]:
df_venue.head()

,Venue ID,Category,Nb of Visitors,Latidude,Longitude
0,49bbd6c0f964a520f4531fe3,Arts & Crafts Store,7,40.719810375488535,-74.00258103213994
1,4a43c0aef964a520c6a61fe3,Bridge,37,40.60679958140643,-74.04416981025437
2,4c5cc7b485a1e21e00d35711,Home (private),1,40.716161684843215,-73.88307005845945
3,4bc7086715a7ef3bef9878da,Medical Center,1,40.7451638,-73.982518775
4,4cf2c5321d18a143951b5cec,Food Truck,4,40.74010382743943,-73.98965835571289


In [8]:
df_venue.describe()

,Venue ID,Category,Nb of Visitors,Latidude,Longitude
count,5000,5000,5000,5000,5000
unique,5000,210,88,3536,3329
top,4dcc652e52b18f1ce69f22ef,Bar,1,40.75280620914171,-73.98887973002674
freq,1,317,2032,26,33


In [9]:
print("We use Data of {} different venues with a total of {} visitors".format(len(df_venue["Venue ID"]), sum(df_venue["Nb of Visitors"])))

We use Data of 5000 different venues with a total of 25909 visitors


#### Since i want to check for good restaurant opportunities, i only need those kind of categories

In [10]:
df_venue_rest = df_venue.loc[df_venue['Category'].str.contains("Restaurant")] #new DF which contains only restaurants
#Restaurants = list(df_venue["Category"].loc[df_venue['Category'].str.contains("Restaurant")]) #list of all restaurants

In [11]:
df_venue_rest.sort_values(by=["Nb of Visitors"], ascending=False, inplace = True)
df_venue_rest.head()

C:\Users\jonas\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Venue ID,Category,Nb of Visitors,Latidude,Longitude
1523,3fd66200f964a5205ae91ee3,American Restaurant,29,40.80991152281341,-74.06785011291504
554,477a3514f964a520214d1fe3,American Restaurant,25,40.750794799423865,-73.99357639021292
672,4d5b3dd522ad2d430a63e695,German Restaurant,23,40.856994,-73.932996
382,4a0ad357f964a520ae741fe3,Mexican Restaurant,23,40.90266453914116,-74.00578886034883
2066,3fd66200f964a520b8ea1ee3,Eastern European Restaurant,21,40.70235639879379,-73.79167762239943


#### Find the most visited Restaurants
In order to find the most visited restaurants i'll use groupby. So for me as a german guy i can see that german resturants are in the top 10 of the most visited resturants, so thats maybe a kind of a business opportunity to open another one, to sell some of those awesom sausages ;-) 

In [14]:
df_restaurants = df_venue_rest.groupby(["Category"]).sum()
df_restaurants.drop(["Venue ID", "Latidude","Longitude"], axis = 1, inplace = True)
df_restaurants.sort_values(by=["Nb of Visitors"], ascending=False, inplace = True)
df_restaurants.head(20)

,Nb of Visitors
Category,
American Restaurant,567
Mexican Restaurant,430
Fast Food Restaurant,169
Italian Restaurant,162
Chinese Restaurant,145
Vegetarian / Vegan Restaurant,114
French Restaurant,103
Sushi Restaurant,102
Asian Restaurant,87


#### Get the locations of the german Restaurants

In [15]:
df_ger_rest = df_venue_rest[df_venue_rest["Category"] =="German Restaurant"]
df_ger_rest

,Venue ID,Category,Nb of Visitors,Latidude,Longitude
672,4d5b3dd522ad2d430a63e695,German Restaurant,23,40.856994,-73.932996
2047,45633154f964a520a53d1fe3,German Restaurant,8,40.66263818239269,-73.90887022018433
2456,4ebf131261af06192af35189,German Restaurant,5,40.687068739999994,-73.975857


In [16]:
locations_ger = df_ger_rest[['Latidude', 'Longitude']] 
locationlist_ger = locations_ger.values.tolist() 
locationlist_ger = [[float(lat),float(lon)] for [lat,lon] in locationlist_ger]

locations = df_venue_rest.iloc[:1000][['Latidude', 'Longitude']]
locationlist = locations.values.tolist() 
locationlist = [[float(lat),float(lon)] for [lat,lon] in locationlist]

In [17]:
map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

for point in range(0,len(locations)):
    folium.CircleMarker(locationlist[point],
                color='blue',
                radius=1,
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7,
                popup=str(df_venue_rest.iloc[point]["Category"])
                 ).add_to(map)
    
for point in range(0, len(locationlist_ger)):
    folium.CircleMarker(locationlist_ger[point],
                color='red',
                radius=2,
                fill=True,
                fill_color='#ff0000',
                fill_opacity=0.7,
                popup=str(df_ger_rest.iloc[point]["Category"])
                 ).add_to(map)
    
map

<p>So in this map of the top 5000 visited restaurants (in blue) and the three german restaurants (in red) we clearly can see that most of the restaurant check-ins are in lower and midtown Manhatten. So the first guess is to open a restaurant here, because the closest german restaurant is in brooklyn.</p>
My approach is to calculate the midpoint between two german restaurants. After that i count how many restaurants are in the neighborhood (e.g. in a circle of a given radius). When there are many restaurants (with a lot nb of visitors) i conclue that there is a certain deman for restaurants, so thats a good location for a new german restaurant

#### Define some function to work with lat - long - coordinates

In [18]:
from math import sin, cos, atan2, asin, sqrt, pi, degrees, radians, pow 


def dist(lat1, long1, lat2, long2):
    r = 6371 #radius of the earth
    p = pi/180
    arg = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((long2-long1)*p))/2
    return 2 * r * asin(sqrt(arg))

#in order to use min() function for our df we need to use the key argument to get the key for the closest datapoints
def midpoint_lat_long(lat1, long1, lat2, long2):
    dLong = radians(long2-long1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    long1 = radians(long1)

    Bx = cos(lat2)*cos(dLong)
    By = cos(lat2)*sin(dLong)
    
    lat3 = atan2(sin(lat1) + sin(lat2), sqrt((cos(lat1) + Bx) * (cos(lat1) + Bx) + By * By)) 
    long3 = long1 + atan2(By, cos(lat1) + Bx);
    
    return [degrees(lat3), degrees(long3)]

def point_in_range(lat1, long1, lat2,long2,r):
    distance = dist(lat1, long1, lat2,long2)
    if distance <=r:
        return True
    else:
        return False

#### Calculate the dist of the german restaurants and count how many restaurants are nearby each midpoint

In [19]:
ger_coords = df_ger_rest[["Venue ID","Latidude", "Longitude"]].values.tolist()
midpoint_data = dict()

for i in range(0,3):
    for j in range(i+1,3):
        ven_ID_1 = ger_coords[i][0]
        lat1 = float(ger_coords[i][1])
        long1 = float(ger_coords[i][2])
        ven_ID_2 = ger_coords[j][0]
        lat2 = float(ger_coords[j][1])
        long2 = float(ger_coords[j][2])
        
        midpoint = midpoint_lat_long(lat1, long1, lat2, long2)
        if (ven_ID_1,ven_ID_2) not in midpoint_data.keys():
            midpoint_data[(ven_ID_1,ven_ID_2)] = midpoint

In [20]:
coord_rest_overall = df_venue_rest[["Latidude","Longitude", "Nb of Visitors"]].values
coord_midpoint = list(midpoint_data.values())


count_midpoint = [0,0,0]
count_visitors_in_radius = [0,0,0]

radius = 4 #2km
for i in range(0,len(coord_midpoint)):
    for j in range(0,len(coord_rest_overall)):
        lat1, long1 = float(coord_midpoint[i][0]), float(coord_midpoint[i][1])
        lat2, long2 = float(coord_rest_overall[j][0]), float(coord_rest_overall[j][1])
        counter = int(coord_rest_overall[j][2])
        dist_mid_rest = dist(lat1, long1, lat2, long2)
        if dist_mid_rest<=radius:
            count_midpoint[i]+=1
            count_visitors_in_radius[i] += counter

In [21]:
map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

for point in range(0,len(locations)):
    folium.CircleMarker(locationlist[point],
                color='blue',
                radius=0.5,
                fill=True,
                fill_color='#3186cc',
                fill_opacity=1.,
                popup=str(df_venue_rest.iloc[point]["Category"])
                 ).add_to(map)
    
for point in range(0, len(locationlist_ger)):
    folium.CircleMarker(locationlist_ger[point],
                color='#FF0000',
                radius=3,
                fill=True,
                fill_color='#FF0000',
                fill_opacity=1.,
                popup=str(df_ger_rest.iloc[point]["Category"])
                 ).add_to(map)

for point in range(0,len(midpoint_data)):
    folium.Circle(list(midpoint_data.values())[point],
                color='#008000',
                radius=4000,
                fill=True,
                fill_color='#008000',
                fill_opacity=0.,
                dash_array='10',
                #popup=str(str(count_midpoint[point]) + "Restaurants are nearby")
                 ).add_to(map)  

for point in range(0,len(midpoint_data)):
    folium.CircleMarker(list(midpoint_data.values())[point],
                color='#008000',
                radius=3,
                fill=True,
                fill_color='#008000',
                fill_opacity=1.,
                popup=str(str(count_midpoint[point]) + " Restaurants are nearby with a total of "+ str(count_visitors_in_radius[point]) + " visitors.")
                 ).add_to(map)    

map

Now that we have out potential location, lets get some more information about the possible competitors, so first lets check if those german restaurants have ratings 

In [22]:
CLIENT_ID = 'Z5Q2ZXXKWSSV1NB1BN2YNVFTU0254NU0WODNSCOA3WQZJTD4' # your Foursquare ID
CLIENT_SECRET = 'OGMV43YS5RYAIUDFRJD21I5ROFFOA0U2QQO0JCFPVROS5UQY' # your Foursquare Secret
ACCESS_TOKEN = 'PUAUMCNZWTL53XCFPN0ZISIPPXBPBGQJ4RTARXQCFLD0RNU1#_=_' # your FourSquare Access Token
VERSION = '20200312'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z5Q2ZXXKWSSV1NB1BN2YNVFTU0254NU0WODNSCOA3WQZJTD4
CLIENT_SECRET:OGMV43YS5RYAIUDFRJD21I5ROFFOA0U2QQO0JCFPVROS5UQY


In [23]:
venue_id = '4d5b3dd522ad2d430a63e695' # 1. German Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [24]:
venue_id = '45633154f964a520a53d1fe3' # 2. German Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [25]:
venue_id = '4ebf131261af06192af35189' # 3. German Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


#### Problem: Data used above is quite old (2012), lets find out, how many german restaurants are in this area in 2020

In [26]:
latitude = "40.77203335214001"  #coords of the most pop midpoint 
longitude = "-73.95445390369426"
search_query = "German"
radius = 5000
VERSION = "20200312"
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Z5Q2ZXXKWSSV1NB1BN2YNVFTU0254NU0WODNSCOA3WQZJTD4&client_secret=OGMV43YS5RYAIUDFRJD21I5ROFFOA0U2QQO0JCFPVROS5UQY&ll=40.77203335214001,-73.95445390369426&v=20200312&query=German&radius=5000&limit=30'

In [27]:
result = requests.get(url).json()

In [32]:
venues = result['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

C:\Users\jonas\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


In [33]:
dataframe.drop(["referralId", "hasPerk", "location.labeledLatLngs", "location.country", "location.cc", "venuePage.id"], axis = 1, inplace = True)
dataframe.rename(columns={"location.distance": "Distance to midpoint","location.lat":"latitude", "location.lng":"longitude", "location.postalCode":"Postal Code", "location.city":"City", "location.state":"State", "location.formattedAddress": "Address", "location.crossStreet": "Street"}, inplace = True)
dataframe.head()

,id,name,categories,location.address,latitude,longitude,Distance to midpoint,Postal Code,City,State,Address,Street
0,4be59180bcef2d7f67f103e5,German Auto Place,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",14-14 Astoria Blvd,40.772196,-73.928402,2196,11102,Astoria,NY,"[14-14 Astoria Blvd, Astoria, NY 11102, United...",NaN
1,54d31df4498e0e2689566ab5,German-american Oktoberfest,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",NaN,40.769751,-73.960593,576,NaN,New York,NY,"[New York, NY, United States]",NaN
2,533b37b8498e4a2a8a0e3c7a,German National Tourist Office,"[{'id': '4f4530164b9074f6e4fb00ff', 'name': 'T...",NaN,40.750070,-73.976865,3090,NaN,New York,NY,"[New York, NY, United States]",NaN
3,4ad0de2df964a52092da20e3,German Evangelical Lutheran Church of St Paul,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",315 W 22nd St,40.745262,-73.999485,4826,10011,New York,NY,"[315 W 22nd St (at 8th Ave), New York, NY 1001...",at 8th Ave
4,55f18d24498e651eec6c5298,The German Residence of Kleft,"[{'id': '50aaa5234b90af0d42d5de12', 'name': 'H...",NaN,40.773847,-73.965443,948,NaN,New York,NY,"[New York, NY, United States]",NaN


In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
dataframe['categories'] = dataframe.apply(get_category_type, axis=1)


In [36]:
data_ger_rest = dataframe[dataframe['categories'].str.contains("Restaurant")]
data_ger_rest

,id,name,categories,location.address,latitude,longitude,Distance to midpoint,Postal Code,City,State,Address,Street
8,5357ee88498ee81170010971,German House,German Restaurant,NaN,40.752988,-73.967148,2374,NaN,New York,NY,"[New York, NY, United States]",NaN
25,3fd66200f964a5207ae51ee3,Rolf's German Restaurant,German Restaurant,281 3rd Ave,40.737974,-73.983516,4514,10010,New York,NY,"[281 3rd Ave (at E 22nd St), New York, NY 1001...",at E 22nd St
26,4d5b3dd522ad2d430a63e695,Hofbräu Bierhaus NYC,German Restaurant,712 3rd Ave,40.752631,-73.973304,2681,10017,New York,NY,"[712 3rd Ave (45th St), New York, NY 10017, Un...",45th St
29,526c2d4011d252b42d12e22e,Germany's Bratwurst,German Restaurant,NaN,40.759687,-73.987746,3125,NaN,New York,NY,"[8th Ave, New York, NY, United States]",8th Ave


In [39]:
New_ger_rest_coords = data_ger_rest[["latitude","longitude"]].values
New_ger_rest_name = data_ger_rest["name"].values
midpoint_data

{('4d5b3dd522ad2d430a63e695', '45633154f964a520a53d1fe3'): [40.75981671917467,
  -73.92091547488207],
 ('4d5b3dd522ad2d430a63e695', '4ebf131261af06192af35189'): [40.77203335214001,
  -73.95445390369426],
 ('45633154f964a520a53d1fe3', '4ebf131261af06192af35189'): [40.67485830032398,
  -73.94235747358412]}

#### Seems like there is a new competitor in town at the moment, lets find out, if hes in our territory

In [58]:
map = folium.Map(location=[40.7128, -74.0060], zoom_start=10)


    
for point in range(0, len(locationlist_ger)):
    folium.CircleMarker(locationlist_ger[point],
                color='#FF0000',
                radius=3,
                fill=True,
                fill_color='#FF0000',
                fill_opacity=1.,
                popup="German Restaurant 2012" 
                 ).add_to(map)

for point in range(0,len(midpoint_data)):
    folium.Circle(list(midpoint_data.values())[point],
                color='#008000',
                radius=4000,
                fill=True,
                fill_color='#008000',
                fill_opacity=0.,
                dash_array='10',
                 ).add_to(map)  

for point in range(0,len(midpoint_data)):
    folium.CircleMarker(list(midpoint_data.values())[point],
                color='#008000',
                radius=3,
                fill=True,
                fill_color='#008000',
                fill_opacity=1.,
                popup=str(str(count_midpoint[point]) + " Restaurants are nearby with a total of "+ str(count_visitors_in_radius[point]) + " visitors.")
                 ).add_to(map)  

for point in range(0,len(New_ger_rest_coords)):
    folium.Circle([New_ger_rest_coords[point][0],New_ger_rest_coords[point][1]],
                color='blue',
                radius=3,
                fill=True,
                fill_color='#3186cc',
                fill_opacity=1.,
                popup="New German Restaurant 2020" 
                 ).add_to(map)


feature_group = folium.FeatureGroup(name='<span style=\\"color: red;\\">Old German Restaurants</span>')
feature_group2 = folium.FeatureGroup(name='<span style=\\"color: blue;\\">New German Restaurants</span>')
feature_group3 = folium.FeatureGroup(name='<span style=\\"color: green;\\">Midpointarea(r=4km)</span>')


map.add_child(feature_group)
map.add_child(feature_group2)
map.add_child(feature_group3)


# turn on layer control
map.add_child(folium.map.LayerControl('topleft', collapsed= False))
    
map

As we can see the "old" German restaurants are closed in 2020, maybe because of the corona situation. Anyhow the "new" German restaurants are in the exakt area we predicted to be the most attractive one with the data of 2012. This show that the analysis and the prediction was correct, based on the data i had/ used. Sadly i didnt find data with Number of Visitors for these 4 restaurants, so im not able to do a prediction like above. 